# 使えるフォントの確認


In [ ]:
from matplotlib import font_manager as fm
name_of_fonts=fm.findSystemFonts()

In [ ]:
import numpy as np
import pandas as pd
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

url = "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000032103844&fileKind=1"

# Read CSV directly from URL
cpi_req = pd.read_csv(url, encoding="shift-jis")

# Remove first 5 rows
cpi = cpi_req.iloc[5:].reset_index(drop=True)

# Rename columns
cpi = cpi.rename(columns={"類・品目": "年月"})
start_m=cpi['年月'][0][:-2]+'-'+cpi['年月'][0][4:]
end_m=cpi['年月'][len(cpi)-1][:-2]+'-'+cpi['年月'][len(cpi)-1][4:]
date_indices=pd.date_range(start_m,end_m,freq='MS')

# Convert columns to numeric
lncpi_TTL=np.log(np.float64(cpi["総合"]))
# Calculate percentage change
dlnCPI = (lncpi_TTL[12:]-lncpi_TTL[:-12]) * 100

# 時系列データの定義
dlnCPI=pd.Series(dlnCPI,index=date_indices[12:])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager as fm
import sys

if 'darwin' in str(sys.implementation) : 
  plt.rcParams['font.family'] = 'Hiragino Sans GB'
elif 'w32' in str(sys.implementation) :
  plt.rcParams['font.family'] = 'MS Gothic'
else :
  plt.rcParams['font.family'] = 'ipaexfont-gothic'

# Plot
plt.close('all')
plt.figure()
plt.plot(dlnCPI, linestyle='-',marker='', color='b')
plt.xlabel("年月")
plt.ylabel("%")
plt.title("総合CPI変化率(対前年比)")
plt.axhline(y=0, linestyle='--', color='gray')
plt.show()

In [ ]:
import pandas as pd
import requests

# Downloading and reading population census data
pop_url = "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000032142402&fileKind=0"
pop_response = requests.get(pop_url)
with open("pop_census.xlsx", "wb") as f:
    f.write(pop_response.content)
pop = pd.read_excel("pop_census.xlsx")

# Downloading and reading labor force survey data
lfs_url = "https://www.e-stat.go.jp/stat-search/file-download?statInfId=000031831358&fileKind=0"
lfs_response = requests.get(lfs_url)
with open("lfs.xlsx", "wb") as f:
    f.write(lfs_response.content)
lfs = pd.read_excel("lfs.xlsx", sheet_name="原数値")

# Removing downloaded files
import os
os.remove("pop_census.xlsx")
os.remove("lfs.xlsx")

# Removing rows and renaming columns for population data
pop = pop.iloc[14:]
pop.columns = ["code", "Pref2000", "RCode2000", "R2000", "Pref2020", "RCode2020", "RName", "total", "male", "female", "total2015", "dN", "r_dN", "r_gender", "area_km2", "density", "HH", "HH_generic", "HH_facility", "HH2015", "dHH", "r_dHH", "HHmem", "HHmem_g", "HHmem_f"]
pop.reset_index(drop=True, inplace=True)
m = pop.shape[1]
pop.iloc[:, 7:m] = pop.iloc[:, 7:m].apply(pd.to_numeric, errors='coerce')

# Removing rows and renaming columns for labor force survey data
lfs = lfs.iloc[15:-13]
lfs.columns = ["Year", "Month", "Month_en", "Blnk", "LF", "LF_m", "LF_f", "Emp", "Emp_m", "Emp_f", "Empee", "Empee_m", "Empee_f", "U", "U_m", "U_f", "NLF", "NLF_m", "NLF_f", "UR", "UR_m", "UR_f"]
lfs.reset_index(drop=True, inplace=True)
m = lfs.shape[1]
lfs.iloc[:, 4:m] = lfs.iloc[:, 4:m].apply(pd.to_numeric, errors='coerce')

# Generate LFMonth sequence
LFS_start_m='1972-07'
for i in range(len(lfs)-1,0,-1) :
  if str(lfs['Year'][i]).find('20')>-1 :
    break
LFS_end_m=str(lfs['Year'][i])+'-'+lfs['Month'][len(lfs)-1].replace('月','')
LFS_date_indices=pd.date_range(LFS_start_m,LFS_end_m,freq='MS')

URate=pd.Series(list(map(float, lfs["UR"])),index=LFS_date_indices)

# Plotting complete unemployment rate
plt.close('all')
plt.figure()
plt.plot(URate, linestyle="-", marker="", color="blue")
plt.title("完全失業率")
plt.ylabel("%")
plt.xlabel("")
plt.show()

# Plotting Phillips Relations
plt.close('all')
plt.figure()
n = len(dlnCPI)
UR = list(map(float, lfs["U"]) / list(map(float, lfs["LF"]) * 100
plt.scatter(UR, dlnCPI[18:n], s=20, alpha=0.5)
plt.title("Phillips Relations")
plt.xlabel("失業率")
plt.ylabel("インフレ率")
plt.show()

In [ ]:
from keras.datasets import mnist
(train_images,train_labels), (test_images,test_labels)=mnist.load_data()

In [ ]:
train_images.shape
len(test_labels)

In [ ]:
from keras import models
from keras import layers
network=models.Sequential()
network.add(layers.Dense(512,activation='relu',input_shape=(28 * 28, )))
network.add(layers.Dense(10,activation='softmax'))